# Feature engineering

In [1]:
import csv
import numpy as np
import pandas as pd
import sklearn

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
df = pd.read_csv("/content/drive/My Drive/MFE230T-Fall2020/class_notebooks/3-Feature_Engineering/loan.csv")
status = ['Fully Paid','Default', 'Charged Off', 'Does not meet the credit policy. Status:Charged Off']
data = df[df.loan_status.isin(status)]

#binary classification
data.loc[:, ('loan_status')]  = data.loan_status.apply(lambda x : +1 if x=='Fully Paid' else -1)


/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [4]:
data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,1277178,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1075269,1311441,5000.0,5000.0,5000.0,36 months,7.90,156.46,A,A4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
data.dtypes

id                               int64
member_id                        int64
loan_amnt                      float64
funded_amnt                    float64
funded_amnt_inv                float64
term                            object
int_rate                       float64
installment                    float64
grade                           object
sub_grade                       object
emp_title                       object
emp_length                      object
home_ownership                  object
annual_inc                     float64
verification_status             object
issue_d                         object
loan_status                      int64
pymnt_plan                      object
url                             object
desc                            object
purpose                         object
title                           object
zip_code                        object
addr_state                      object
dti                            float64
delinq_2yrs              

## Data transformations

In [6]:
print(data.purpose.unique())

['credit_card' 'car' 'small_business' 'other' 'wedding'
 'debt_consolidation' 'home_improvement' 'major_purchase' 'medical'
 'moving' 'vacation' 'house' 'renewable_energy' 'educational']


In [7]:
#dti = deb to income
data = data[['loan_amnt','dti','purpose', 'loan_status']][:10000]
data.head(10)

,loan_amnt,dti,purpose,loan_status
0,5000.0,27.65,credit_card,1
1,2500.0,1.00,car,-1
2,2400.0,8.72,small_business,1
3,10000.0,20.00,other,1
5,5000.0,11.20,wedding,1
7,3000.0,5.35,car,1
8,5600.0,5.55,small_business,-1
9,5375.0,18.08,other,-1
10,6500.0,16.12,debt_consolidation,1
11,12000.0,10.78,debt_consolidation,1


In [8]:
from sklearn.preprocessing import LabelEncoder
categ = df.purpose.unique()
le = LabelEncoder()
le.fit(categ)
print(list(le.classes_))
data.purpose = data['purpose'].to_frame().apply(le.fit_transform)
print(data.head(10))

['car', 'credit_card', 'debt_consolidation', 'educational', 'home_improvement', 'house', 'major_purchase', 'medical', 'moving', 'other', 'renewable_energy', 'small_business', 'vacation', 'wedding']
    loan_amnt    dti  purpose  loan_status
0      5000.0  27.65        1            1
1      2500.0   1.00        0           -1
2      2400.0   8.72       10            1
3     10000.0  20.00        8            1
5      5000.0  11.20       12            1
7      3000.0   5.35        0            1
8      5600.0   5.55       10           -1
9      5375.0  18.08        8           -1
10     6500.0  16.12        2            1
11    12000.0  10.78        2            1


In [9]:
one = pd.get_dummies(data.purpose, prefix='p')
print(one.head(10))

    p_0  p_1  p_2  p_3  p_4  p_5  p_6  p_7  p_8  p_9  p_10  p_11  p_12
0     0    1    0    0    0    0    0    0    0    0     0     0     0
1     1    0    0    0    0    0    0    0    0    0     0     0     0
2     0    0    0    0    0    0    0    0    0    0     1     0     0
3     0    0    0    0    0    0    0    0    1    0     0     0     0
5     0    0    0    0    0    0    0    0    0    0     0     0     1
7     1    0    0    0    0    0    0    0    0    0     0     0     0
8     0    0    0    0    0    0    0    0    0    0     1     0     0
9     0    0    0    0    0    0    0    0    1    0     0     0     0
10    0    0    1    0    0    0    0    0    0    0     0     0     0
11    0    0    1    0    0    0    0    0    0    0     0     0     0


In [10]:
data[one.columns] =  one
print(data.head(10))

    loan_amnt    dti  purpose  loan_status  p_0  p_1  p_2  p_3  p_4  p_5  p_6  \
0      5000.0  27.65        1            1    0    1    0    0    0    0    0   
1      2500.0   1.00        0           -1    1    0    0    0    0    0    0   
2      2400.0   8.72       10            1    0    0    0    0    0    0    0   
3     10000.0  20.00        8            1    0    0    0    0    0    0    0   
5      5000.0  11.20       12            1    0    0    0    0    0    0    0   
7      3000.0   5.35        0            1    1    0    0    0    0    0    0   
8      5600.0   5.55       10           -1    0    0    0    0    0    0    0   
9      5375.0  18.08        8           -1    0    0    0    0    0    0    0   
10     6500.0  16.12        2            1    0    0    1    0    0    0    0   
11    12000.0  10.78        2            1    0    0    1    0    0    0    0   

    p_7  p_8  p_9  p_10  p_11  p_12  
0     0    0    0     0     0     0  
1     0    0    0     0     0   

In [11]:
del data['purpose']
print(data.head(10))

    loan_amnt    dti  loan_status  p_0  p_1  p_2  p_3  p_4  p_5  p_6  p_7  \
0      5000.0  27.65            1    0    1    0    0    0    0    0    0   
1      2500.0   1.00           -1    1    0    0    0    0    0    0    0   
2      2400.0   8.72            1    0    0    0    0    0    0    0    0   
3     10000.0  20.00            1    0    0    0    0    0    0    0    0   
5      5000.0  11.20            1    0    0    0    0    0    0    0    0   
7      3000.0   5.35            1    1    0    0    0    0    0    0    0   
8      5600.0   5.55           -1    0    0    0    0    0    0    0    0   
9      5375.0  18.08           -1    0    0    0    0    0    0    0    0   
10     6500.0  16.12            1    0    0    1    0    0    0    0    0   
11    12000.0  10.78            1    0    0    1    0    0    0    0    0   

    p_8  p_9  p_10  p_11  p_12  
0     0    0     0     0     0  
1     0    0     0     0     0  
2     0    0     1     0     0  
3     1    0     0  

## Text feature extraction

In [12]:
text = df.title[:10000]

In [13]:
print(text.head(10))

0                                 Computer
1                                     bike
2                     real estate business
3                                 personel
4                                 Personal
5    My wedding loan I promise to pay back
6                                     Loan
7                          Car Downpayment
8     Expand Business & Buy Debt Portfolio
9              Building my credit history.
Name: title, dtype: object


In [14]:
from sklearn.feature_extraction.text import CountVectorizer

#transforming words to vectors of counts
count_vect = CountVectorizer(stop_words='english')

text = text.apply(lambda x : str(x).lower())

#create a sparse dictionary (matrix) for each title
dictionary = count_vect.fit_transform(text)

In [15]:
print(dictionary[:10])

  (0, 430)	1
  (1, 240)	1
  (2, 301)	1
  (2, 720)	1
  (2, 1475)	1
  (3, 1373)	1
  (4, 1370)	1
  (5, 1341)	1
  (5, 1434)	1
  (5, 1110)	1
  (5, 1909)	1
  (6, 1110)	1
  (7, 663)	1
  (7, 326)	1
  (8, 1406)	1
  (8, 589)	1
  (8, 307)	1
  (8, 730)	1
  (8, 301)	1
  (9, 939)	1
  (9, 536)	1
  (9, 294)	1


In [16]:
print(count_vect.get_feature_names())

[u'00', u'000', u'02', u'05', u'07', u'08', u'0809healthbills', u'081011', u'09', u'10', u'100', u'10000', u'101', u'10k', u'11', u'1166', u'12', u'120', u'12142011', u'13k', u'14', u'14400', u'14k', u'15', u'150', u'16', u'162', u'16k', u'16kloan', u'17', u'18', u'18mos', u'19', u'190e', u'1941', u'1948', u'1964', u'1968', u'1969', u'1975', u'1976', u'1995', u'1999', u'1fb', u'1k', u'1st', u'2001', u'2003', u'2005', u'2007', u'2010', u'2011', u'2011loan', u'2011loanconsol', u'2011payoff', u'2011q4', u'2012', u'2014', u'2015', u'20k', u'21k', u'21sept2011', u'22', u'23', u'24', u'2400', u'25', u'250', u'26', u'27', u'29', u'2d', u'2nd', u'30', u'300', u'3100', u'34', u'350', u'35k', u'36', u'3900', u'3k', u'3rd', u'40', u'401', u'401k', u'403b', u'42k', u'4x4', u'5000', u'502countdown', u'50th', u'5200', u'53', u'56', u'57apr', u'5k', u'5th', u'60', u'6000', u'60m', u'62', u'67', u'6x6', u'70', u'720', u'72003yr', u'73', u'7750', u'7k', u'800', u'84', u'85', u'8k', u'900', u'95', u'98'

In [17]:
from sklearn.feature_extraction.text import TfidfTransformer

# Transforms a count matrix to a normalized tf or tf-idf representation
# tf-idf(d, t) = tf(t) * idf(d, t)
# idf(d, t) = log [ n / df(d, t) ] + 1 
tf_transformer = TfidfTransformer().fit(dictionary)
dictionary_tf = tf_transformer.transform(dictionary)

print(dictionary_tf[:10])

  (0, 430)	1.0
  (1, 240)	1.0
  (2, 1475)	0.6648571530246645
  (2, 720)	0.6541149451557592
  (2, 301)	0.3606918415986367
  (3, 1373)	1.0
  (4, 1370)	1.0
  (5, 1909)	0.41446696046382286
  (5, 1434)	0.8094977789966827
  (5, 1341)	0.3739301687601286
  (5, 1110)	0.1819525030724755
  (6, 1110)	1.0
  (7, 663)	0.8715825361105424
  (7, 326)	0.49024879678293437
  (8, 1406)	0.5993411234306438
  (8, 730)	0.5738074019569468
  (8, 589)	0.13898827823972584
  (8, 307)	0.4609733101442337
  (8, 301)	0.28234933871170526
  (9, 939)	0.6989648069142245
  (9, 536)	0.24570945151319779
  (9, 294)	0.6716212207282052


In [18]:
tfidf = dictionary_tf.todense()
dd = pd.DataFrame(tfidf)
print(dd.head(10))

   0     1     2     3     4     5     6     7     8     9     ...  1948  \
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
1   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
2   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
3   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
4   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
5   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
6   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
7   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
8   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
9   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   

   1949  1950  1951  1952  1953  1954  1955  1956  1957  
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
1   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0

In [19]:
print(text.iloc[5])

my wedding loan i promise to pay back


In [20]:
print(dictionary[5,:])

  (0, 1110)	1
  (0, 1341)	1
  (0, 1434)	1
  (0, 1909)	1


In [21]:
print(count_vect.inverse_transform(dictionary[5,:]))

[array([u'loan', u'pay', u'promise', u'wedding'], dtype='<U34')]


In [22]:
print(dictionary_tf[5,:])

  (0, 1909)	0.41446696046382286
  (0, 1434)	0.8094977789966827
  (0, 1341)	0.3739301687601286
  (0, 1110)	0.1819525030724755
